# Statistical Inference

## Import libraries

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import plotly.graph_objects as go
from IPython.display import display
from sklearn.metrics import mean_squared_error as mse
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
sns.set()

In [19]:
df_full = pd.read_csv("outputs/df_full.csv")
df_full.drop('Unnamed: 0', inplace=True, axis=1)
df_full

,gdp,gnp,real_gdp,real_gdp_per_capita,net_exports,gni,govt_spending,consumer_spending,private_domestic_investment,cpi,consumer_oil_price,ir,unemployment_rate,Close,Volume,ClosePrev,CloseNext
0,308.153,309.760,2340.112,15398.0,-0.740,307.413,600.663,200.505,1.247,24.203,11.267,1.61,4.6,18.180000,2.760741e+07,17.186667,18.570000
1,319.945,321.554,2384.920,15623.0,-0.154,319.791,643.100,197.946,1.289,24.693,11.500,1.75,4.2,18.570000,2.260370e+07,18.180000,19.816667
2,336.000,337.537,2417.311,15769.0,0.177,336.177,711.537,209.207,1.296,25.697,11.700,1.75,3.5,19.816667,3.068148e+07,18.570000,21.620000
3,344.090,345.973,2459.196,15979.0,1.943,346.033,806.376,204.942,1.332,25.947,11.933,1.75,3.1,21.620000,3.020555e+07,19.816667,21.636667
4,351.385,353.381,2509.880,16234.0,3.742,355.127,895.015,207.616,1.385,25.933,11.933,1.75,3.2,21.636667,2.172778e+07,21.620000,22.980000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,19477.444,19649.442,17258.205,52031.0,-538.876,18938.568,3378.132,12989.729,519.850,256.418,219.570,0.25,13.0,2921.443333,5.985144e+10,3136.440000,3019.010000
280,21138.574,21365.412,18560.774,55933.0,-725.723,20412.851,3360.238,14293.832,539.864,259.438,232.403,0.25,8.8,3019.010000,6.714349e+10,2921.443333,3378.143333
281,21477.597,21728.223,18767.778,56533.0,-798.431,20679.166,3356.030,14467.611,561.269,260.879,234.862,0.25,6.8,3378.143333,5.204018e+10,3019.010000,3549.220000
282,22038.226,22273.060,19055.655,57405.0,-872.540,21165.686,3390.921,15005.444,576.340,263.525,274.983,0.25,6.2,3549.220000,5.152288e+10,3378.143333,3832.760000


## Import Results from LSTM Model

In [20]:
res_df = pd.read_csv("outputs/res_df.csv")
res_df.drop('Unnamed: 0', inplace=True, axis=1)
display(res_df.describe())

,MSE
count,20.000000
mean,78305.750298
std,200200.272392
min,7912.070815
25%,11466.811390
50%,13556.713018
75%,15937.655031
max,663816.384904


## Remove Outliers

In [21]:
# Remove outliers
Q3 = res_df.describe().loc['75%'].at['MSE']
Q1 = res_df.describe().loc['25%'].at['MSE']
IQR = Q3-Q1
res_df = res_df[(res_df['MSE']>=Q1-1.5*IQR) & (res_df['MSE']<=Q3+1.5*IQR)]
print("Remaining values after removing outliers:")
display(res_df)
display(res_df.describe())

# res_df.to_csv('results/lstm_results.csv')

Remaining values after removing outliers:


,MSE
0,13784.256021
2,15787.038724
3,10290.518741
4,13896.817178
6,9242.089766
7,11858.908939
8,12752.546323
9,13776.037066
10,13333.109139
12,16389.503952


,MSE
count,17.000000
mean,12683.515497
std,2689.647638
min,7912.070815
25%,10290.518741
50%,13333.109139
75%,13896.817178
max,17438.499120


## Import Results from Regression Model

In [22]:
res_df2 = pd.read_csv("outputs/res_df2.csv")
res_df2.drop('Unnamed: 0', inplace=True, axis=1)

res_df2.index.name = 'random_state'
print("Values before removing outliers:")
display(res_df2)
display(res_df2.describe())

q1_var = res_df2.describe().loc['25%'].at['R^2']
q3_var = res_df2.describe().loc['75%'].at['R^2']
iqr_var = q3_var - q1_var

q1_MSE = res_df2.describe().loc['25%'].at['MSE train']
q3_MSE = res_df2.describe().loc['75%'].at['MSE train']
iqr_MSE = q3_MSE - q1_MSE

q1_test = res_df2.describe().loc['25%'].at['MSE test']
q3_test = res_df2.describe().loc['75%'].at['MSE test']
iqr_test = q3_test - q1_test

state = res_df2[(res_df2['R^2'] >= q1_var-1.5*iqr_var) & (res_df2['R^2'] <= q3_var+1.5*iqr_var)
               & (res_df2['MSE train'] >= q1_MSE-1.5*iqr_MSE) & (res_df2['MSE train'] <= q3_MSE+1.5*iqr_MSE)
              & (res_df2['MSE test'] >= q1_test-1.5*iqr_test) & (res_df2['MSE test'] <= q3_test+1.5*iqr_test)].index.tolist()
res_df2 = res_df2.iloc[state]
print("Remaining values after removing outliers:")
display(res_df2)
display(res_df2.describe())

# res_df2.to_csv('results/regression_results.csv')

Values before removing outliers:


,R^2,MSE train,MSE test
random_state,,,
0,0.996022,2769.926266,5601.987580
1,0.995072,3294.501151,4527.341277
2,0.995329,3260.694419,3478.071381
3,0.995441,3377.506674,3109.400642
4,0.994611,3526.101231,5141.723797
...,...,...,...
996,0.995473,3218.404259,3812.822810
997,0.996039,2908.112518,8199.956137
998,0.995831,3152.769411,4209.897918


,R^2,MSE train,MSE test
count,1001.000000,1001.000000,1001.000000
mean,0.995617,3027.751487,6866.822803
std,0.000390,266.979482,9527.849522
min,0.994224,2216.784908,1856.656870
25%,0.995369,2849.322603,4308.950830
50%,0.995619,3036.449672,5293.613809
75%,0.995894,3224.041344,6880.869654
max,0.996724,3816.484236,210460.027045


Remaining values after removing outliers:


,R^2,MSE train,MSE test
random_state,,,
0,0.996022,2769.926266,5601.987580
1,0.995072,3294.501151,4527.341277
2,0.995329,3260.694419,3478.071381
3,0.995441,3377.506674,3109.400642
4,0.994611,3526.101231,5141.723797
...,...,...,...
996,0.995473,3218.404259,3812.822810
997,0.996039,2908.112518,8199.956137
998,0.995831,3152.769411,4209.897918


,R^2,MSE train,MSE test
count,925.000000,925.000000,925.000000
mean,0.995626,3049.707013,5449.483991
std,0.000370,249.369933,1718.635184
min,0.994609,2372.924735,1950.452411
25%,0.995384,2871.914814,4225.930251
50%,0.995627,3052.072597,5141.795051
75%,0.995898,3235.809881,6346.456432
max,0.996612,3775.167902,10517.531853


## Visualize Regressian Median Test Results (with error bars) 

In [23]:
y = pd.DataFrame(df_full['CloseNext'])
X = pd.DataFrame(df_full[['real_gdp', 'consumer_spending', 'gnp', 'gdp', 'private_domestic_investment']])

# Find best train-test-split based on median R^2 or MSE
option = 'MSE' # 'R^2' or 'MSE' ('MSE' refers to 'MSE train')

med = 0
state = 0
if (option == 'R^2'):
    med = res_df2.describe().loc['50%'].at['R^2']
    state = res_df2['R^2'].sub(med).abs().idxmin()
elif (option == 'MSE'):
    med = res_df2.describe().loc['50%'].at['MSE train']
    state = res_df2['MSE train'].sub(med).abs().idxmin()
print(f'random_state = {state}')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=state)

poly = PolynomialFeatures(degree=2)
poly_reg = LinearRegression()

# train model
X_ = poly.fit_transform(X_train)
X_ = np.delete(X_,(1),axis=1)
poly_reg.fit(X_, y_train)
y_pred = poly_reg.predict(X_)
# test model
X_2 = poly.fit_transform(X_test)
X_2 = np.delete(X_2,(1),axis=1)
y_pred2 = poly_reg.predict(X_2)
# whole model (train+test)
X_3 = poly.fit_transform(X)
X_3 = np.delete(X_3,(1),axis=1)
y_pred3 = poly_reg.predict(X_3)

print(f'Explained Variance: {poly_reg.score(X_, y_train)}')
print(f'Train Set MSE: {mse(y_train, y_pred)}')
print(f'Test Set MSE: {mse(y_test, y_pred2)}')

fig = go.Figure()

fig.add_trace(go.Scatter(x=df_full.index, y=y['CloseNext'], name="Actual Price"))

#fig.add_trace(go.Scatter(x=y.index, y=pd.Series(np.squeeze(y_pred3)), name="Predicted Price (TRAIN + TEST)"))

error = np.array(y_test) - np.array(y_pred2)
fig.add_trace(go.Scatter(
        x=y_test.index, y=pd.Series(np.squeeze(y_pred2)),
        error_y=dict(
            type='data',
            symmetric=False,
            array=error,
            visible=True),
        mode='markers',
        name="Predicted Price (TEST)"))

fig.update_layout(
    title_text='### Mean Square Error :' + str(mse(y_test, y_pred2).round(2)) + ' ###'
)

fig.update_xaxes(title_text="<b>Date</b>")
fig.update_yaxes(title_text="<b>Price</b>")

fig.show()

random_state = 470
Explained Variance: 0.9955722341906255
Train Set MSE: 3052.072597061365
Test Set MSE: 4725.39673011144


In [24]:
display(poly_reg.coef_)
display(poly_reg.intercept_)

array([[ 0.00000000e+00, -2.95172045e+00,  2.51244273e+01,
        -2.40731433e+01,  2.74803672e+01,  9.56141749e-06,
         7.41735813e-04, -4.59552721e-03,  4.21227914e-03,
        -3.07246285e-03, -1.27092166e-03,  1.28371057e-02,
        -1.27146592e-02,  3.90385024e-02, -2.31291457e-03,
         5.04047686e-04, -6.74394414e-02,  2.21475307e-03,
         4.37620280e-02, -3.93878299e-04]])

array([105.7372506])

## Hypothesis Testing (Welch T Test) - Comparing the 2 Model's Reliability

In [25]:
reg_df = pd.read_csv("results/regression_results.csv")
reg = reg_df['MSE test']
lstm = pd.read_csv("results/lstm_results.csv")
lstm.drop('Unnamed: 0', inplace=True, axis=1)
from scipy import stats
p_value = stats.ttest_ind(reg, lstm, equal_var = False)
print("p-value: ", p_value[1][1])

p-value:  5.756720798857384e-09


Let h0= "both means are equal", h1= "lstm mean is higher than reg", p value of test statistic is 5.756720798857384e-09, so at a 99% confidence interval we reject the null hypothesis.

# Intelligent Decision

## Conclusion

#### Regression model performs much better than LSTM because it doesn't assume that the past indicates the future. <br> It puts more emphasis on understanding the relationship between economic situation and sentiments to stock prices.

## Recommendations

1) Use a predictive model that takes both past trends and current economic situations as predictors <br> 
2) Use weekly data instead of quarterly data for more precise results i.e. quarterly/monthly data (e.g. GDP) can be converted to weekly/daily data by repeating the same values within the same quarter/month <br> 
3) Bring in market sentiment indicators i.e NLP tools that scrapes and convert news headlines to measurable market/economic parameters <br>
4) To prevent overfitting, ensure to de-trend features and only select top-most correlated ones <br>